### Import Libraries

In [9]:
import re
import pandas as pd
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.chunk import RegexpParser
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
string.punctuation

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### Preprocessing

In [10]:
# read data (data yang digunakan adalah data sample 500 baris agar pemrosesannya tidak lama)

data = pd.read_excel('data/new_paten.xlsx')
data = data[["title", "abstract"]]
data.head()

,title,abstract
0,A kind of artificial intelligence skin and its...,The invention discloses a kind of artificial i...
1,A kind of image-recognizing method based on na...,The invention discloses a kind of image-recogn...
2,The pedestrian detection method merged based o...,The present invention relates to a kind of ped...
3,A kind of Knowledge Base based on natural lang...,The invention discloses a kind of Knowledge Ba...
4,Text emotion analysis method and device,The invention discloses a kind of text emotion...


In [11]:
# add . in the end of title
def add_period(text):
    if pd.notna(text):
        return text + ". "
    else:
        return text

# Apply the function to the 'title' column
data['title'] = data['title'].apply(add_period)

# merge title and abstract column
data['title_and_abstract'] = data['title'].fillna('') + data['abstract']

# Print the updated DataFrame
data.head()

,title,abstract,title_and_abstract
0,A kind of artificial intelligence skin and its...,The invention discloses a kind of artificial i...,A kind of artificial intelligence skin and its...
1,A kind of image-recognizing method based on na...,The invention discloses a kind of image-recogn...,A kind of image-recognizing method based on na...
2,The pedestrian detection method merged based o...,The present invention relates to a kind of ped...,The pedestrian detection method merged based o...
3,A kind of Knowledge Base based on natural lang...,The invention discloses a kind of Knowledge Ba...,A kind of Knowledge Base based on natural lang...
4,Text emotion analysis method and device.,The invention discloses a kind of text emotion...,Text emotion analysis method and device. The i...


In [12]:
# check missing value

print(data.isna().sum())

title                 0
abstract              0
title_and_abstract    0
dtype: int64


In [13]:
# # remove missing value (tidak perlu karena kolom title and abstract tidak memiliki missing value)

# data.dropna(inplace=True)
# data.head()

In [14]:
# lowering text
data['title_and_abstract']= data['title_and_abstract'].apply(lambda x: x.lower())

# Print the updated DataFrame
data['title_and_abstract'].head()

0    a kind of artificial intelligence skin and its...
1    a kind of image-recognizing method based on na...
2    the pedestrian detection method merged based o...
3    a kind of knowledge base based on natural lang...
4    text emotion analysis method and device. the i...
Name: title_and_abstract, dtype: object

In [15]:
# Function to tokenize each sentence, tokenize each word from sentence, then remove punctuation and sopword
def tokenize_and_remove_punctuation(text):
    # Define stopword
    stop_words = set(stopwords.words('english'))

    # Tokenize each sentence
    sentences = re.split(r'(?<=[.!?,;])\s+', text)
    
    # Tokenize each word in each sentence, remove punctuation, and remove stopword
    tokenized_sentences = []
    for sentence in sentences:
        tokens = word_tokenize(sentence)
        # Remove punctuation
        tokens = [token for token in tokens if token not in string.punctuation]
        # Remove stopword
        tokens = [token for token in tokens if token not in stop_words]
        tokenized_sentences.append(tokens)
    return tokenized_sentences

# Tokenize each sentence and remove punctuation & stopword
data['tokenized'] = data['title_and_abstract'].apply(tokenize_and_remove_punctuation)

# Print the updated DataFrame
print(data['tokenized'][0])

[['kind', 'artificial', 'intelligence', 'skin', 'method', 'detection', 'humiture', 'pressure'], ['invention', 'discloses', 'kind', 'artificial', 'intelligence', 'skin'], ['includes', 'signal', 'acquisition', 'part', 'circuit', 'part'], ['signal', 'acquisition', 'part', 'point', 'includes', 'humidity', 'sensitive', 'material', 'layer'], ['first', 'electrode', 'layer'], ['insulating', 'layer'], ['second', 'electrode', 'lay'], ['thermistor', 'material', 'layer'], ['3rd', 'electrode', 'layer'], ['piezoelectric', 'material', 'layer', '4th', 'electrode', 'layer', 'set', 'gradually', 'top', 'bottom'], ['circuit', 'part', 'includes', 'front-end', 'circuit', 'back-end', 'circuit.the', 'invention', 'also', 'discloses', 'method', 'using', 'above-mentioned', 'artificial', 'intelligence', 'skin', 'detection', 'humiture', 'pressure.the', 'configuration', 'present', 'invention', 'simple'], ['easy', 'miniaturization', 'integrated'], ['test', 'method', 'succinct'], ['rigorous.the', 'present', 'inventio

### POS Tag

In [16]:
# Function to perform POS tagging on each token
def pos_tag_tokens(tokenized_sentences):
    pos_tagged_sentences = []
    for sentence_tokens in tokenized_sentences:
        pos_tags = pos_tag(sentence_tokens)
        pos_tagged_sentences.append(pos_tags)
    return pos_tagged_sentences

# Perform POS tagging on tokenized sentences
data['pos_tagged'] = data['tokenized'].apply(pos_tag_tokens)

# Print the updated DataFrame
print(data['pos_tagged'][0])

[[('kind', 'NN'), ('artificial', 'JJ'), ('intelligence', 'NN'), ('skin', 'NN'), ('method', 'NN'), ('detection', 'NN'), ('humiture', 'NN'), ('pressure', 'NN')], [('invention', 'NN'), ('discloses', 'NNS'), ('kind', 'NN'), ('artificial', 'JJ'), ('intelligence', 'NN'), ('skin', 'NN')], [('includes', 'VBZ'), ('signal', 'JJ'), ('acquisition', 'NN'), ('part', 'NN'), ('circuit', 'NN'), ('part', 'NN')], [('signal', 'JJ'), ('acquisition', 'NN'), ('part', 'NN'), ('point', 'NN'), ('includes', 'VBZ'), ('humidity', 'NN'), ('sensitive', 'JJ'), ('material', 'NN'), ('layer', 'NN')], [('first', 'RB'), ('electrode', 'VBZ'), ('layer', 'NN')], [('insulating', 'VBG'), ('layer', 'NN')], [('second', 'JJ'), ('electrode', 'NN'), ('lay', 'VBD')], [('thermistor', 'NN'), ('material', 'NN'), ('layer', 'NN')], [('3rd', 'CD'), ('electrode', 'NN'), ('layer', 'NN')], [('piezoelectric', 'JJ'), ('material', 'NN'), ('layer', 'NN'), ('4th', 'CD'), ('electrode', 'NN'), ('layer', 'NN'), ('set', 'VBN'), ('gradually', 'RB'), (

In [17]:
# Function to lemmatize tokens
def lemmatize_tokens(pos_tagged_sentences):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentences = []
    for sentence_tags in pos_tagged_sentences:
        lemmatized_tokens = [(lemmatizer.lemmatize(token, pos=get_wordnet_pos(pos_tag)), pos_tag) for token, pos_tag in sentence_tags]
        lemmatized_sentences.append(lemmatized_tokens)
    return lemmatized_sentences

# Function to map POS tags to WordNet POS tags
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return nltk.corpus.wordnet.NOUN  # Default to noun if POS tag not recognized

# Lemmatize the tokens in 'pos_tagged_sentences' column
data['lemmatized'] = data['pos_tagged'].apply(lemmatize_tokens)

# Print the updated DataFrame
print(data['lemmatized'][0])

[[('kind', 'NN'), ('artificial', 'JJ'), ('intelligence', 'NN'), ('skin', 'NN'), ('method', 'NN'), ('detection', 'NN'), ('humiture', 'NN'), ('pressure', 'NN')], [('invention', 'NN'), ('discloses', 'NNS'), ('kind', 'NN'), ('artificial', 'JJ'), ('intelligence', 'NN'), ('skin', 'NN')], [('include', 'VBZ'), ('signal', 'JJ'), ('acquisition', 'NN'), ('part', 'NN'), ('circuit', 'NN'), ('part', 'NN')], [('signal', 'JJ'), ('acquisition', 'NN'), ('part', 'NN'), ('point', 'NN'), ('include', 'VBZ'), ('humidity', 'NN'), ('sensitive', 'JJ'), ('material', 'NN'), ('layer', 'NN')], [('first', 'RB'), ('electrode', 'VBZ'), ('layer', 'NN')], [('insulate', 'VBG'), ('layer', 'NN')], [('second', 'JJ'), ('electrode', 'NN'), ('lay', 'VBD')], [('thermistor', 'NN'), ('material', 'NN'), ('layer', 'NN')], [('3rd', 'CD'), ('electrode', 'NN'), ('layer', 'NN')], [('piezoelectric', 'JJ'), ('material', 'NN'), ('layer', 'NN'), ('4th', 'CD'), ('electrode', 'NN'), ('layer', 'NN'), ('set', 'VBN'), ('gradually', 'RB'), ('top

### POS Chunk

In [18]:
# define pattern
pattern_raharjana = r'''Chunk: {<VB\w*>+<.|IN|CC|PRP\w*>*<DT>*<NN\w*|RB>+}'''
pattern_webb = r'''Chunk: {<VB\w*><NN\w*>}'''
pattern_johann = r'''Chunk: {<VB\w*><NN\w*>|<VB\w*><PRP\w*><NN\w*>|<VB\w*><NN\w*><NN\w*>|<VB\w*><JJ\w*><NN\w*>|<VB\w*><DT><NN\w*>|<VB\w*><NN\w*><IN><NN\w*>|<VB\w*><IN><JJ\w*><NN\w*>|<VB\w*><PRP\w*><JJ\w*><NN\w*>}'''

def extract(tagged_texts, pattern):
    chunk_parser = RegexpParser(pattern)
    chunks = []
    for tagged_text in tagged_texts:
        tree = chunk_parser.parse(tagged_text)
        tree = extract_chunks(tree)
        tree = trees_to_tuples(tree)
        chunks.append(tree)
    chunks = [item for sublist in chunks for item in sublist]
    return chunks

def extract_chunks(tree):
    chunks = []
    
    if isinstance(tree, nltk.Tree):
        if tree.label() != 'S':  # Exclude sentence-level chunks if any
            chunks.append(tree)
        for subtree in tree:
            chunks.extend(extract_chunks(subtree))
    
    return chunks

def trees_to_tuples(tree_list):
    tuple_list = [tuple(leaf[0] for leaf in tree.leaves()) for tree in tree_list]
    return tuple_list

# Apply chunking and extraction to the 'tagged_sentences' column
data['pattern_webb'] = data['lemmatized'].apply(lambda x: extract(x, pattern_webb))
data['pattern_raharjana'] = data['lemmatized'].apply(lambda x: extract(x, pattern_raharjana))
data['pattern_johann'] = data['lemmatized'].apply(lambda x: extract(x, pattern_johann))

# Print the resulting DataFrame with chunked data
data['pattern_webb']

0         [(include, humidity), (electrode, layer), (ins...
1         [(distribute, weight), (establish, information...
2         [(relate, kind), (use, model), (disaggregated,...
3         [(put, default), (find, father), (define, sub-...
4         [(dictionary；based, language), (extend, form),...
                                ...                        
138780    [(improve, performance), (recognise, traffic),...
138781    [(perform, computer), (comprise, step), (acqui...
138782    [(comprise, head), (comprise, head), (submit, ...
138783    [(base, machine), (predict, method), (base, ma...
138784    [(microwave, source), (microwave, radiation), ...
Name: pattern_webb, Length: 138785, dtype: object

### Export Result

In [19]:
# hasile

patent_task = data[["title_and_abstract", "pattern_webb", "pattern_raharjana", "pattern_johann"]]
patent_task.to_excel('export_result/paten_3pattern_lengkap.xlsx', index=False)